In [70]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset , DataLoader
import pandas as pd

import os

from transformers import BertTokenizer

from transformers import default_data_collator

# **seed 고정**

In [71]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

GPU

In [72]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# **Hugging face dateset가져오기**

In [73]:
# Hugging Face datasets 라이브러리 설치 (설치되지 않은 경우)
!pip install datasets

# GLUE 데이터셋 불러오기
from datasets import load_dataset

# MRPC (Microsoft Research Paraphrase Corpus) 태스크 로드 예시
dataset = load_dataset("glue", "mrpc")

In [74]:
# 데이터셋 정보 출력
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})


sentence1 , sentence2의 동의어 관계 판별 ,

---
label : 1 -> 동의어 관계
label : 0 -> 동의어 관계 아님

---
idx : 고유 id



In [75]:
# 훈련, 검증 데이터 예시 출력
print(f"Train Example: {dataset['train'][0]}")
print(f"Validation Example: {dataset['validation'][0]}")

Train Example: {'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0}
Validation Example: {'sentence1': "He said the foodservice pie business doesn 't fit the company 's long-term growth strategy .", 'sentence2': '" The foodservice pie business does not fit our long-term growth strategy .', 'label': 1, 'idx': 9}


# **토큰화**

In [76]:
# 2. BERT 토크나이저 불러오기 (사전 학습된 BERT 모델 사용)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [77]:
# 3. 토큰화 함수 정의
def tokenize_function(examples):
    # sentence1과 sentence2를 함께 토큰화 (Padding, Truncation 처리)
    return tokenizer(examples["sentence1"],
                     examples["sentence2"],
                     padding="max_length",
                     truncation=True,
                     max_length=16)

In [78]:
# 4. 데이터셋에 토큰화 적용
tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [79]:
# 5. 토큰화된 데이터 예시 출력
print(tokenized_dataset["train"][0])

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .', 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .', 'label': 1, 'idx': 0, 'input_ids': [101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 102, 7727, 2000, 2032, 2004, 2069, 1000, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


구조 :

---

(sentence1 원문 , sentence2 원문 , label , idx , input_ids1 & 2 , token_type_ids , attention_mask)

---
token_type_ids : 두 문장(문장1, 문장2)을 구분하기 위한 인코딩 (BERT에서 사용),

---

attention_mask : 패딩된 부분을 무시하도록 마스킹 처리 (1은 유효한 토큰, 0은 패딩)


# **배치화**

In [80]:
# 데이터셋을 DataLoader에 넣기
train_loader = train_loader = DataLoader(tokenized_dataset["train"], batch_size=256, collate_fn=default_data_collator)

In [81]:
# 3. 배치 반복 및 크기 확인
for batch in train_loader:
    print(batch.keys())  # 딕셔너리의 키 확인 (예: input_ids, attention_mask, label)

    input_ids = batch['input_ids']
    print(f"Input IDs shape: {len(input_ids)}")
    print(f"Input IDs shape: {len(input_ids[0])}")

    attention_mask = batch['attention_mask']
    print(f"Attention Mask shape: {len(attention_mask)}")
    print(f"Attention Mask shape: {len(attention_mask[0])}")
    break  # 첫 배치만 확인

dict_keys(['labels', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'])
Input IDs shape: 256
Input IDs shape: 16
Attention Mask shape: 256
Attention Mask shape: 16


# **필드 추출**

In [82]:
# 5. DataLoader에서 배치 추출 및 Encoder에 입력 예시
for batch in train_loader:
    # 필요한 텐서 추출
    input_ids = batch['input_ids']
    # 어디 문장 소속인지 여부를 표시하는 텐서 추가
    token_type_ids = batch['token_type_ids']
    attention_mask = batch['attention_mask']
    label = batch['labels']

    print(f"Batch size: {len(input_ids)}, Sequence length: {len(input_ids[0])}")
    print(f"label : {len(label)}")

Batch size: 256, Sequence length: 16
label : 256
Batch size: 256, Sequence length: 16
label : 256
Batch size: 256, Sequence length: 16
label : 256
Batch size: 256, Sequence length: 16
label : 256
Batch size: 256, Sequence length: 16
label : 256
Batch size: 256, Sequence length: 16
label : 256
Batch size: 256, Sequence length: 16
label : 256
Batch size: 256, Sequence length: 16
label : 256
Batch size: 256, Sequence length: 16
label : 256
Batch size: 256, Sequence length: 16
label : 256
Batch size: 256, Sequence length: 16
label : 256
Batch size: 256, Sequence length: 16
label : 256
Batch size: 256, Sequence length: 16
label : 256
Batch size: 256, Sequence length: 16
label : 256
Batch size: 84, Sequence length: 16
label : 84


# **데이터 처리**

In [83]:
# batch는 dictionary 형태이므로 필요한 필드만 따로 추출해야한다
# token embeddings + segment embeddings + positional embeddings

# **Encoder**

# **hyperparameter**

In [119]:
num_heads = 16
embedding_dim = 16 * num_heads

In [120]:
class MultiHeadAttention(nn.Module):
  def __init__(self, num_heads , embedding_dim , input_size = 16):
    super().__init__()
    self.embedding_dim = embedding_dim
    self.num_heads = num_heads
    self.head_dim = embedding_dim // num_heads
    self.input_size = input_size

    self.q = nn.Linear(embedding_dim , embedding_dim)
    self.k = nn.Linear(embedding_dim , embedding_dim)
    self.v = nn.Linear(embedding_dim , embedding_dim)

    self.fc = nn.Linear(embedding_dim , embedding_dim)

  def go(self , x):

    batch_size = x.shape[0]

    q = self.q(x) # batch_num , sequence_length , embed_size : 16 , 16 , 128
    k = self.k(x)
    v = self.v(x)

    q = q.view(batch_size , -1 , self.num_heads , self.head_dim).transpose(1,2) # batch , num_head, 16 , 16*8//8 = 16
    k = k.view(batch_size , -1 , self.num_heads , self.head_dim).transpose(1,2)
    v = v.view(batch_size , -1 , self.num_heads , self.head_dim).transpose(1,2)

    # Scaled Dot-Product Attention
    attention_score = (q @ k.transpose(3,2)) / (self.head_dim ** 0.5)
    attention_score = torch.softmax(attention_score, dim=-1)
    attention = torch.matmul(attention_score , v)
    attention = attention.transpose(1,2).contiguous().view(batch_size , -1 , self.embedding_dim) # batch , 8 , sequence_length , 16*8//8 = 16
    attention = attention.reshape(x.shape[0] , x.shape[1] , x.shape[2])
    output = self.fc(attention)

    return output # batch , sequence_length , 16*8

# **인풋의 길이가 전부 16으로 일정하므로 padding을 추가하지 않는다**

In [121]:
class Pre_process(nn.Module):
    def __init__(self ,num_heads , embedding_dim , vocab_size = 40000 , max_length = 16, dropout=0.1):
        super().__init__()
        self.embed_size = embedding_dim
        self.max_length = max_length
        self.word_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.position_embedding = nn.Parameter(torch.randn(1 , max_length , embedding_dim))
        self.dropout = nn.Dropout(dropout)
    def run(self , input , token_type_ids , mask = None):
        batch_num , seq_length = input.shape
        z = self.word_embedding(input) # z.shape : batch_num , sequence_length , embedding_dim = 16 , 16 , 128
        positional_embed = self.position_embedding.expand(batch_num , self.max_length , self.embed_size)
        # 단어 임베딩 + position + 문장 소속(binary)
        out = z + positional_embed + token_type_ids.unsqueeze(2).expand(batch_num,self.max_length,self.embed_size)
        out = self.dropout(out)
        return out # out.shape : batch_num , sequence_length , embed_size = 16 , 16 , 128

x -> Encoder -> y -> multiheadattention -> z -> Encoder_block

# **Encoder block**

In [122]:
class Encoder_Block(nn.Module):
  def __init__(self, num_heads , embed_size , dropout = 0.1):
    super().__init__()

    self.relu = nn.ReLU()

    self.norm1 = nn.LayerNorm(embed_size)
    self.norm2 = nn.LayerNorm(1)

    self.l1 = nn.Linear(embed_size , embed_size)
    self.l2 = nn.Linear(embed_size , 1)
    self.l3 = nn.Linear(16 , 1)

    self.pre_process = Pre_process(num_heads , embed_size)
    self.attention = MultiHeadAttention(num_heads , embed_size)

    self.dropout = nn.Dropout(0.1)

  def update(self , input , token_type_ids):

    c = self.pre_process.run(input , token_type_ids)
    z = self.attention.go(c)

    z = z + c
    # print(z.shape) , [16, 16, 128]
    z1 = self.norm1(z)

    z = self.l1(z1)
    z = self.relu(z)

    #skip
    z = z1 + z  # z.shape : 16 ,16 , 128

    z = self.l2(z) # 16 ,16 , 1
    z = z.reshape(z.shape[0] , 16)
    z = self.l3(z) # 16 , 1

    z = self.norm2(z) # 16 , 1

    return z # 16 , 1

Model 설계

In [123]:
model = Encoder_Block(num_heads , embedding_dim)

# **아키텍쳐**

In [124]:
#model = model.to(device)
optimizer =  torch.optim.Adam(model.parameters(), lr=0.01)

In [125]:
cnt = 0
loss_history = []
test_loss_history = []
bce_loss = nn.BCEWithLogitsLoss()

epochs = 100

model.train()
for epoch in range(epochs):
  for batch in train_loader:

    # 필요한 텐서 추출
    input_ids = batch['input_ids']
    #input_ids = torch.stack(input_ids)

    # 어디 문장 소속인지 여부를 표시하는 텐서 추가
    token_type_ids = batch['token_type_ids']
    #token_type_ids = torch.stack(token_type_ids)

    #label
    labels = batch['labels'].unsqueeze(1)

    #print(model.device)
    output = model.update(input_ids , token_type_ids)
    loss = bce_loss(output, labels.float())

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # loss history append
    loss_history.append(loss.item())

  #scheduler_linear.step()
  print("epoch : {} , loss : {}".format(epoch, loss))

epoch : 0 , loss : 0.6647324562072754
epoch : 1 , loss : 0.6406077146530151
epoch : 2 , loss : 0.6233289837837219
epoch : 3 , loss : 0.6117141842842102
epoch : 4 , loss : 0.6042519807815552
epoch : 5 , loss : 0.5996034145355225
epoch : 6 , loss : 0.5967675447463989
epoch : 7 , loss : 0.5950652956962585
epoch : 8 , loss : 0.5940607786178589
epoch : 9 , loss : 0.5934818983078003
epoch : 10 , loss : 0.5931591987609863
epoch : 11 , loss : 0.5929873585700989
epoch : 12 , loss : 0.5929015278816223
epoch : 13 , loss : 0.592862069606781
epoch : 14 , loss : 0.5928464531898499
epoch : 15 , loss : 0.5928419232368469
epoch : 16 , loss : 0.592842161655426
epoch : 17 , loss : 0.5928438901901245
epoch : 18 , loss : 0.5928460359573364


KeyboardInterrupt: 

초기 , num_heads = 8 , lr = 0.001

---


epoch99 : 0.594

learning_rate from 0.001 to 0.01

---
epcoh99 : 0.592


lr : 0.005 , num_heads = 16